In [4]:
import numpy as np
import matplotlib.pyplot as plt
import os.path
import os
from scipy.interpolate import interp1d
from scipy.signal import butter,filtfilt
from scipy.optimize import curve_fit
from scipy import odr
from scipy import linalg
import itertools

# material parameters

In [5]:
# highest order polynomial
order = 6

# density in g/cm^3
rho = 1

# mass in g
m = 0.0089
m=1

# dimensions of sample in m
Lx = 1.4
Ly = 1.2
Lz = 1.9

# density in kg/m^3
# rho = m*1e-3 / (Lx*Ly*Lz)


# number of basis functions
N = int( (order+1) * (order+2) * (order+3) / 6 )

# give the elastic tensor

In [6]:
# elastic constants in GPa
c11 = 2*48.16944 + 42.47116
c22 = 2*48.16944 + 42.47116
c33 = 194.58691
c12 = 42.47116
c13 = 14.29768
c23 = 14.29768
c44 = 45.20971
c55 = 45.20971
c66 = 48.16944

c11 = 5
c22 = 5
c33 = 5

c12 = .5
c13 = .5
c23 = .5

c44 = 1
c55 = 1
c66 = 1



C = np.zeros([3,3,3,3])
# this is not done yet - I have to symmetrize and all
C[0,0,0,0] = c11
C[1,1,1,1] = c22
C[2,2,2,2] = c33

C[0,0,1,1] = c12
C[1,1,0,0] = c12

C[2,2,0,0] = c13
C[0,0,2,2] = c13

C[1,1,2,2] = c23
C[2,2,1,1] = c23

C[0,1,0,1] = c44
C[1,0,0,1] = c44
C[0,1,1,0] = c44
C[1,0,1,0] = c44

C[0,2,0,2] = c55
C[2,0,0,2] = c55
C[0,2,2,0] = c55
C[2,0,2,0] = c55

C[1,2,1,2] = c66
C[2,1,2,1] = c66
C[2,1,1,2] = c66
C[1,2,2,1] = c66

# create the basis functions for the displacement and their derivatives

In [7]:
# phi: basis functions with order <= highest order
# dphi: the derivatives of phi with respect to x, y, z
# they are labelled by their power in x, y, z
phi = []
dphidx = []
dphidy = []
dphidz = []

for i in np.arange(order+1):
    for j in  np.arange(order+1-i):
        for k in np.arange(order+1-i-j):
            phi.append([i, j, k])
            dphidx.append([i-1, j, k])
            dphidy.append([i, j-1, k])
            dphidz.append([i, j, k-1])
phi = np.array(phi)
dphidx = np.array(dphidx)
dphidy = np.array(dphidy)
dphidz = np.array(dphidz)

dphidx[dphidx < 0] = 0
dphidy[dphidy < 0] = 0
dphidz[dphidz < 0] = 0

# calculating the kinetic energy

In [8]:
# this gives phi_lambda*phi_lambda^prime as a tensor
phi_squaredx = np.add.outer(phi[:,0], phi[:,0])
phi_squaredy = np.add.outer(phi[:,1], phi[:,1])
phi_squaredz = np.add.outer(phi[:,2], phi[:,2])

In [9]:
# here I integrate the phi^2 term over the sample

# this is because integrating a polynomial means increasing the exponent by 1
exponent_add = np.ones((N,N))

# integrating the x, y, and z components individually
integral_kin_x = ( (Lx/2)**(phi_squaredx+exponent_add) - (-Lx/2)**(phi_squaredx+exponent_add) ) / (phi_squaredx+exponent_add)
integral_kin_y = ( (Ly/2)**(phi_squaredy+exponent_add) - (-Ly/2)**(phi_squaredy+exponent_add) ) / (phi_squaredy+exponent_add)
integral_kin_z = ( (Lz/2)**(phi_squaredz+exponent_add) - (-Lz/2)**(phi_squaredz+exponent_add) ) / (phi_squaredz+exponent_add)
integral_kinetic = integral_kin_x * integral_kin_y * integral_kin_z # multiplying the different integrals because that was the original problem

# this is now where the final kinetic energy is calculated
# Ekin is the matrix integral_kinetic multiplied with a delta fct with unrelated indices
# this is mapped to a 3*len(integral_kinetic) x 3*len(integral_kinetic) matrix, where the diagonal elements are the matrix
# integral_kinetic and everything else is zero
Ekin = np.zeros( [3*N, 3*N ] )
Ekin[:N, :N] = integral_kinetic
Ekin[N:2*N, N:2*N] = integral_kinetic
Ekin[2*N:3*N, 2*N:3*N] = integral_kinetic
Ekin = Ekin * rho

# calculate potential energy

In [10]:
# this gives phi_lambda*phi_lambda^prime as a tensor
dphidx_dphidx_x = np.add.outer(dphidx[:,0], dphidx[:,0])
dphidx_dphidx_y = np.add.outer(dphidx[:,1], dphidx[:,1])
dphidx_dphidx_z = np.add.outer(dphidx[:,2], dphidx[:,2])
dphidx_dphidx_factor = np.outer(phi[:,0], phi[:,0])

dphidy_dphidy_x = np.add.outer(dphidy[:,0], dphidy[:,0])
dphidy_dphidy_y = np.add.outer(dphidy[:,1], dphidy[:,1])
dphidy_dphidy_z = np.add.outer(dphidy[:,2], dphidy[:,2])
dphidy_dphidy_factor = np.outer(phi[:,1], phi[:,1])

dphidz_dphidz_x = np.add.outer(dphidz[:,0], dphidz[:,0])
dphidz_dphidz_y = np.add.outer(dphidz[:,1], dphidz[:,1])
dphidz_dphidz_z = np.add.outer(dphidz[:,2], dphidz[:,2])
dphidz_dphidz_factor = np.outer(phi[:,2], phi[:,2])

dphidx_dphidy_x = np.add.outer(dphidx[:,0], dphidy[:,0])
dphidx_dphidy_y = np.add.outer(dphidx[:,1], dphidy[:,1])
dphidx_dphidy_z = np.add.outer(dphidx[:,2], dphidy[:,2])
dphidx_dphidy_factor = np.outer(phi[:,0], phi[:,1])

dphidx_dphidz_x = np.add.outer(dphidx[:,0], dphidz[:,0])
dphidx_dphidz_y = np.add.outer(dphidx[:,1], dphidz[:,1])
dphidx_dphidz_z = np.add.outer(dphidx[:,2], dphidz[:,2])
dphidx_dphidz_factor = np.outer(phi[:,0], phi[:,2])

dphidy_dphidz_x = np.add.outer(dphidy[:,0], dphidz[:,0])
dphidy_dphidz_y = np.add.outer(dphidy[:,1], dphidz[:,1])
dphidy_dphidz_z = np.add.outer(dphidy[:,2], dphidz[:,2])
dphidy_dphidz_factor = np.outer(phi[:,1], phi[:,2])

dphi_dphi_x = np.array( [dphidx_dphidx_x, dphidy_dphidy_x, dphidz_dphidz_x, dphidx_dphidy_x, dphidx_dphidz_x, dphidy_dphidz_x] )
dphi_dphi_y = np.array( [dphidx_dphidx_y, dphidy_dphidy_y, dphidz_dphidz_y, dphidx_dphidy_y, dphidx_dphidz_y, dphidy_dphidz_y] )
dphi_dphi_z = np.array( [dphidx_dphidx_z, dphidy_dphidy_z, dphidz_dphidz_z, dphidx_dphidy_z, dphidx_dphidz_z, dphidy_dphidz_z] )
factor = np.array( [dphidx_dphidx_factor, dphidy_dphidy_factor, dphidz_dphidz_factor, dphidx_dphidz_factor, dphidx_dphidz_factor, dphidy_dphidz_factor] )

In [11]:
integral_pot_x = ( (Lx/2)**(dphi_dphi_x+exponent_add) - (-Lx/2)**(dphi_dphi_x+exponent_add) ) / (dphi_dphi_x+exponent_add)
integral_pot_y = ( (Ly/2)**(dphi_dphi_y+exponent_add) - (-Ly/2)**(dphi_dphi_y+exponent_add) ) / (dphi_dphi_y+exponent_add)
integral_pot_z = ( (Lz/2)**(dphi_dphi_z+exponent_add) - (-Lz/2)**(dphi_dphi_z+exponent_add) ) / (dphi_dphi_z+exponent_add)

integral_potential = integral_pot_x * integral_pot_y * integral_pot_z * factor

In [12]:
# integral potential is organized as: xx-0, yy-1, zz-2, xy-3, xz-4, yz-5
potential_energy_11  = C[0,0,0,0]*integral_potential[0] + C[0,0,0,1]*integral_potential[3] + C[0,0,0,2]*integral_potential[4] + C[0,1,0,0]*integral_potential[3] + C[0,1,0,1] *integral_potential[1] + C[0,1,0,2]*integral_potential[5] + C[0,2,0,0]*integral_potential[4] + C[0,2,0,1]*integral_potential[5] + C[0,2,0,2]*integral_potential[3]

potential_energy_22  = C[1,0,1,0]*integral_potential[0] + C[1,0,1,1]*integral_potential[3] + C[1,0,1,2]*integral_potential[4] + C[1,1,1,0]*integral_potential[3] + C[1,1,1,1] *integral_potential[1] + C[1,1,1,2]*integral_potential[5] + C[1,2,1,0]*integral_potential[4] + C[1,2,1,1]*integral_potential[5] + C[1,2,1,2]*integral_potential[3]

potential_energy_33  = C[2,0,2,0]*integral_potential[0] + C[2,0,2,1]*integral_potential[3] + C[2,0,2,2]*integral_potential[4] + C[2,1,2,0]*integral_potential[3] + C[2,1,2,1] *integral_potential[1] + C[2,1,2,2]*integral_potential[5] + C[2,2,2,0]*integral_potential[4] + C[2,2,2,1]*integral_potential[5] + C[2,2,2,2]*integral_potential[3]

potential_energy_12  = C[0,0,1,0]*integral_potential[0] + C[0,0,1,1]*integral_potential[3] + C[0,0,1,2]*integral_potential[4] + C[0,1,1,0]*integral_potential[3] + C[0,1,1,1] *integral_potential[1] + C[0,1,1,2]*integral_potential[5] + C[0,2,1,0]*integral_potential[4] + C[0,2,1,1]*integral_potential[5] + C[0,2,1,2]*integral_potential[3]

potential_energy_13  = C[0,0,2,0]*integral_potential[0] + C[0,0,2,1]*integral_potential[3] + C[0,0,2,2]*integral_potential[4] + C[0,1,2,0]*integral_potential[3] + C[0,1,2,1] *integral_potential[1] + C[0,1,2,2]*integral_potential[5] + C[0,2,2,0]*integral_potential[4] + C[0,2,2,1]*integral_potential[5] + C[0,2,2,2]*integral_potential[3]

potential_energy_23  = C[1,0,2,0]*integral_potential[0] + C[1,0,2,1]*integral_potential[3] + C[1,0,2,2]*integral_potential[4] + C[1,1,2,0]*integral_potential[3] + C[1,1,2,1] *integral_potential[1] + C[1,1,2,2]*integral_potential[5] + C[1,2,2,0]*integral_potential[4] + C[1,2,2,1]*integral_potential[5] + C[1,2,2,2]*integral_potential[3]

In [13]:
Epot = np.zeros( [3*N, 3*N ] )
Epot[:N, :N] = 0.5 * potential_energy_11
Epot[N:2*N, N:2*N] = 0.5 * potential_energy_22
Epot[2*N:3*N, 2*N:3*N] = 0.5 * potential_energy_33

Epot[:N, N:2*N] = potential_energy_12
Epot[:N, 2*N:3*N] = potential_energy_13
Epot[N:2*N, 2*N:3*N] = potential_energy_13

Epot = Epot + np.transpose(Epot)

In [14]:
np.shape(Epot)

(252, 252)

In [19]:
b = linalg.eigh(Epot, Ekin, eigvals_only= True)

In [20]:
import numpy as np 
from numpy import array
from scipy import linalg as LA

N = 6
V = (1.4, 1.2, 1.9)
rho = 1.

C = np.zeros([3, 3, 3, 3])
compress, shear, ax = 5., 1. , .5
for k in range(3):
    C[k,k,k,k] = compress
C[1,2,1,2]=C[2,1,1,2]=C[1,2,2,1]=shear
C[1,0,1,0]=C[0,1,1,0]=C[1,0,0,1]=shear
C[0,2,0,2]=C[2,0,0,2]=C[0,2,2,0]=shear
C[0,0,1,1]=C[1,1,0,0]=ax
C[0,0,2,2]=C[2,2,0,0]=ax
C[2,2,1,1]=C[1,1,2,2]=ax

basis = {}
idx = 0
for l in range(N+1):
    for m in range(N+1):
        for n in range(N+1):
            if l + m + n > N: continue
            for ii in range(3):
                basis[idx] = [l, m, n, ii]
                idx += 1
            
E = np.zeros([idx, idx])
G = np.zeros([idx, idx])

for i in range(idx):
    for j in range(i, idx):
        b1, b2 = basis[i], basis[j]
        pow_sum =array([b1[0]+b2[0], b1[1]+b2[1], b1[2]+b2[2]])
        vol_int = (V[0]**(pow_sum[0]+1) + V[1]**(pow_sum[1]+1) + V[2]**(pow_sum[2]+1))/((pow_sum[0]+1)*(pow_sum[1]+1)*(pow_sum[2]+1))
        if b1[-1]==b2[-1]: 
            E[i, j] = rho * vol_int
            if i != j: E[j,i] = E[i,j]
        tmp = 0
        for k in range(3):
            if pow_sum[k] == 1: tmp += 0
            else: tmp += b1[k]*b2[k] * (pow_sum[k] + 1) * C[b1[-1], k, b2[-1], k] / (V[k]**2 * (pow_sum[k] - 1))
            for l in range(3):
                if l == k or pow_sum[l]==0 or pow_sum[k]==0: pass
                else: tmp += b1[k]*b2[l]*(pow_sum[k]+1)*(pow_sum[l]+1)*C[b1[-1], k, b2[-1], l] / (V[k]*V[l]*pow_sum[k]*pow_sum[l])
        G[i, j] = vol_int * tmp 
        if i != j: G[j, i] = G[i,j]

w = LA.eigh(G, E, eigvals_only= True)

print (w)

[-3.20412679e+01 -2.45713290e+01 -2.27743134e+01 -2.02741365e+01
 -1.84580494e+01 -1.80091391e+01 -1.68747240e+01 -1.36342435e+01
 -1.26159568e+01 -1.20320250e+01 -1.07157932e+01 -9.98956468e+00
 -9.26600984e+00 -8.95928250e+00 -8.12863674e+00 -7.32033469e+00
 -6.97351732e+00 -6.42005730e+00 -5.87427759e+00 -5.39281932e+00
 -4.54140325e+00 -4.15026065e+00 -3.81916546e+00 -3.71384744e+00
 -3.19397979e+00 -2.97886141e+00 -2.76395671e+00 -2.38996944e+00
 -2.10721809e+00 -1.67041916e+00 -1.60161748e+00 -1.54105572e+00
 -1.32902762e+00 -1.03139955e+00 -1.00214223e+00 -7.45553369e-01
 -7.09702693e-01 -6.74602237e-01 -6.35887396e-01 -6.04880052e-01
 -5.61848464e-01 -3.61146719e-01 -3.18176058e-01 -2.56488861e-01
 -8.85995841e-02  0.00000000e+00  0.00000000e+00  0.00000000e+00
  8.99072560e-01  1.39346360e+00  2.15082657e+00  2.55648164e+00
  2.74240632e+00  3.37057200e+00  3.43014632e+00  3.76020331e+00
  4.28274687e+00  4.62781904e+00  5.10364874e+00  5.97775292e+00
  6.39676645e+00  6.44071

In [21]:
b

array([-6.78751325e+02, -6.63254437e+02, -4.50436437e+02, -4.28658940e+02,
       -2.69429379e+02, -2.64190022e+02, -2.59349056e+02, -2.51197461e+02,
       -1.90369281e+02, -1.75017790e+02, -1.33530414e+02, -1.29007622e+02,
       -1.23402453e+02, -1.19037278e+02, -9.60704887e+01, -7.08309572e+01,
       -6.82327886e+01, -6.77891262e+01, -5.36321485e+01, -5.17829381e+01,
       -4.48095217e+01, -4.05780834e+01, -3.17984294e+01, -3.07929848e+01,
       -2.80309686e+01, -2.67295329e+01, -2.32280418e+01, -2.22524557e+01,
       -2.07504795e+01, -1.39528461e+01, -1.29529470e+01, -1.02358381e+01,
       -6.75160194e+00, -6.29587307e+00, -3.72241813e+00, -2.31422838e+00,
       -1.72374720e+00, -5.32260936e-01, -1.92526153e-01, -5.34857388e-02,
       -2.92174333e-02, -3.61709386e-14,  0.00000000e+00,  3.42925961e-15,
        8.16027518e-05,  2.23874292e-03,  5.74344908e-02,  8.84074948e-02,
        6.67385093e-01,  1.01025605e+00,  2.57919978e+00,  2.94901997e+00,
        3.41356097e+00,  